In [ ]:
import numpy as np
import pandas as pd 
import time
import requests
import lxml
from lxml.html import fromstring
from fake_useragent import UserAgent

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

ua = UserAgent()
url = "주소"
headers = {'User-Agent' : ua.random}

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

# Change Parameter

In [ ]:
######################## 검색 키워드 입력 ######################## 여기만 수정
KEYWORD = ['드라이브스루', '자동차 와이퍼']

# URL 크롤링

In [ ]:
# 데이터프레임 초기화
df = pd.DataFrame(columns=['url'])
cnt = 0
Bool = True

# url 접근
driver = webdriver.Chrome('chromedriver_91.exe', chrome_options=options) # 크롬 드라이버 다운로드후 본인 경로로 줍니다.
base_url = 'https://www.bobaedream.co.kr/search'
driver.get(base_url)
time.sleep(2)

# 커뮤니티로 이동 - 일부러 time을 많이 주었습니다. 반복문을 도는 것도 아니며, 1번만 접근하면 되기에 안정적으로 시행하는게 중요하다고 판단하였습니다.
driver.find_element_by_class_name('keyword').send_keys(KEYWORD) #검색어 입력
driver.find_element_by_class_name('keyword').send_keys(Keys.ENTER)
time.sleep(3)
driver.find_elements_by_xpath('/html/body/div/div[3]/div[1]/ul/li[5]/a')[0].click()
time.sleep(3)

# 크롤링
while Bool:

    # 스크롤 내리기
    last_element = driver.find_elements_by_xpath('/html/body/div/div[3]/div[2]/div/div[2]')[0]
    driver.execute_script("arguments[0].scrollIntoView(true);", last_element)

    # 정보 수집 (pageid)
    elements = driver.find_elements_by_class_name('search_Community')[0].find_elements_by_tag_name('li')
    url_list = [element.find_elements_by_tag_name('a')[0].get_attribute('href') for element in elements]
    df = pd.concat([df, pd.DataFrame(url_list, columns=['url'])])
    page_num = int(driver.find_elements_by_xpath('/html/body/div/div[3]/div[2]/div/div[2]/strong')[0].text)
    cnt += 1
    print(f'{cnt}페이지 완료')
    
    # 다 됐으면 다음페이지 이동
    try:
        if page_num <= 5:
            
            if page_num % 5 == 1:
                driver.find_elements_by_xpath('/html/body/div/div[3]/div[2]/div/div[2]/a[1]')[0].click()

            elif page_num % 5 == 2:
                driver.find_elements_by_xpath('/html/body/div/div[3]/div[2]/div/div[2]/a[3]')[0].click()

            elif page_num % 5 == 3:
                driver.find_elements_by_xpath('/html/body/div/div[3]/div[2]/div/div[2]/a[4]')[0].click()

            elif page_num % 5 == 4:
                driver.find_elements_by_xpath('/html/body/div/div[3]/div[2]/div/div[2]/a[5]')[0].click()

            elif page_num % 5 == 0:
                driver.find_elements_by_xpath('/html/body/div/div[3]/div[2]/div/div[2]/a[6]')[0].click()
                
        else:
            if page_num % 5 == 1:
                driver.find_elements_by_xpath('/html/body/div/div[3]/div[2]/div/div[2]/a[3]')[0].click()

            elif page_num % 5 == 2:
                driver.find_elements_by_xpath('/html/body/div/div[3]/div[2]/div/div[2]/a[4]')[0].click()

            elif page_num % 5 == 3:
                driver.find_elements_by_xpath('/html/body/div/div[3]/div[2]/div/div[2]/a[5]')[0].click()

            elif page_num % 5 == 4:
                driver.find_elements_by_xpath('/html/body/div/div[3]/div[2]/div/div[2]/a[6]')[0].click()

            elif page_num % 5 == 0:
                driver.find_elements_by_xpath('/html/body/div/div[3]/div[2]/div/div[2]/a[7]')[0].click()
            
        time.sleep(3)
        
        
        
    except:
        Bool = False
        print(f'{cnt}페이지에서 {cnt+1}로 넘어가는 버튼이 존재하지 않는가요? 그렇다면 수집이 완료되었습니다.')
        
df = df.reset_index(drop=True)

# 제목 및 본문 크롤링

In [ ]:
for idx in range(df.shape[0]):
    
    url = df.loc[idx, 'url']
    
    try:
        res = requests.get(url, headers=headers)
        res.raise_for_status()
        res.encoding = "utf-8-sig"
        parser = fromstring(res.text)

        title = parser.xpath('//*[@id="print_area"]/div[1]/dl/dt/strong/text()')[0]
        text = parser.xpath('//*[@id="print_area"]/div[2]/div')[0].text_content()

        df.loc[idx, ['제목', '내용']] = [title, text]
    
    except:
        res = requests.get(url, headers=headers)
        res.raise_for_status()
        res.encoding = "utf-8-sig"
        parser = fromstring(res.text)

        title = parser.xpath('//*[@id="print_area"]/div[1]/dl/dt/strong/text()')[0]
        text = parser.xpath('//*[@id="print_area"]/div[2]/div')[0].text_content()

        df.loc[idx, ['제목', '내용']] = [title, text]
        
    if (idx+1) % 10 == 0:
        print(f'{idx+1}번째 url 완료!!')
        
print(f'{KEYWORD} 저장완료!')
df